<a href="https://colab.research.google.com/github/balandongiv/mmocr_tutorial/blob/main/how_to_load_base_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMOCR Tutorial

Welcome to MMOCR! This is the **unofficial** colab tutorial for using MMOCR. In this tutorial, you will learn how to

- Make config

# Install MMOCR

### Install Dependencies 

In [ ]:

%cd ..
# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

# Install mmdetection
!pip install mmdet

# # Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements.txt
!pip install -v -e .

!pip install wget

### Check Installed Dependencies Versions

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
import os
from pathlib import Path

import wget

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

%cd /mmocr/


# Load exisiting config




In [9]:



!gdown --folder https://drive.google.com/drive/folders/1Lk7O4bg3oX6JRVEzVptf3P_hFFQSfxNK?usp=sharing -O custom_cfg

Retrieving folder list
Processing file 17sx7GvwGSvU0EiilnlX0BE5hTpfhCUJc custom_config.py
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=17sx7GvwGSvU0EiilnlX0BE5hTpfhCUJc
To: /mmocr/custom_cfg/custom_config.py
100% 2.25k/2.25k [00:00<00:00, 4.80MB/s]
Download completed


In [ ]:
import os

_base_ = [
    '/mmocr/configs/_base_/default_runtime.py',
    '/mmocr/configs/_base_/recog_models/sar.py',
    '/mmocr/configs/_base_/schedules/schedule_adam_step_5e.py',
    '/mmocr/configs/_base_/recog_pipelines/sar_pipeline.py',
]

dataset_type = 'OCRDataset'

# Location where the annotation and crop images are being stored
root='/mmocr/tests/data/ocr_toy_dataset'


img_prefix =os.path.join(root,'imgs')


train_anno_file1 = os.path.join(root,'label.txt')



train_datasets1 = dict(
    type='OCRDataset',
    img_prefix=img_prefix,
    ann_file=train_anno_file1,
    loader=dict(
        type='AnnFileLoader',
        repeat=20,
        file_format='txt',
        parser=dict(
            type='LineStrParser',
            keys=['filename', 'text'],
            keys_idx=[0, 1],
            separator=' ')),
    pipeline=None,
    test_mode=False)


train_datasets1 = dict(
    type='OCRDataset',
    img_prefix=img_prefix,
    ann_file=train_anno_file1,
    loader=dict(
        type='AnnFileLoader',
        repeat=20,
        file_format='txt',
        parser=dict(
            type='LineStrParser',
            keys=['filename', 'text'],
            keys_idx=[0, 1],
            separator=' ')),
    pipeline=None,
    test_mode=False)




# For time being, assume the setting for training and validation is the same, though practically we can separate both testing sets



train_list = [train_datasets1]

# For demonstration purpose, we assume the train and validation list is the same
test_list = [train_datasets1]


train_pipeline = {{_base_.train_pipeline}}
test_pipeline = {{_base_.test_pipeline}}

data = dict(
    samples_per_gpu=64,
    workers_per_gpu=2,
    val_dataloader=dict(samples_per_gpu=1),
    test_dataloader=dict(samples_per_gpu=1),
    train=dict(
        type='UniformConcatDataset',
        datasets=train_list,
        pipeline=train_pipeline),
    val=dict(
        type='UniformConcatDataset',
        datasets=test_list,
        pipeline=test_pipeline),
    test=dict(
        type='UniformConcatDataset',
        datasets=test_list,
        pipeline=test_pipeline))

evaluation = dict(interval=1, metric='acc')

Let create cfg

In [10]:
from mmcv import Config

cfg = Config.fromfile('/mmocr/custom_cfg/custom_config.py')

In [12]:
print(f'Config:\n{cfg.pretty_text}')

Config:
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
label_convertor = dict(
    type='AttnConvertor', dict_type='DICT90', with_unknown=True)
model = dict(
    type='SARNet',
    backbone=dict(type='ResNet31OCR'),
    encoder=dict(
        type='SAREncoder', enc_bi_rnn=False, enc_do_rnn=0.1, enc_gru=False),
    decoder=dict(
        type='ParallelSARDecoder',
        enc_bi_rnn=False,
        dec_bi_rnn=False,
        dec_do_rnn=0,
        dec_gru=False,
        pred_dropout=0.1,
        d_k=512,
        pred_concat=True),
    loss=dict(type='SARLoss'),
    label_convertor=dict(
        type='AttnConvertor', dict_type='DICT90', with_unknown=True),
    max_seq_len=30)
optimizer = dict(type='Adam', lr=0.001)
optimizer_config = dict(grad_clip=None)
lr_config = dict(policy='step', step=[3, 4])
runner = dict